In [48]:
#Import librairies
import cv2
import dlib 
import numpy as np
import os 
import matplotlib.pyplot as plt
from tqdm import tqdm
import copy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import torch
from tqdm import tqdm
from typing import Optional, Callable
from sklearn.metrics import accuracy_score, f1_score
from sklearn.covariance import LedoitWolf
import json
from collections.abc import Mapping
from torchvision.transforms import v2
import pandas as pd
# from image_segmentation import *
%load_ext autoreload
%autoreload 2
torch.manual_seed(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
file_name = r"./coin-detector-model/coin-detector-2.svm"
detector = dlib.simple_object_detector(file_name)

In [52]:
from PIL import Image
from model import ResNetModel
possible_value = ["5CHF", "2CHF", "1CHF", "0.5CHF", "0.2CHF", "0.1CHF", "0.05CHF", "2EUR", "1EUR", "0.5EUR", "0.2EUR", "0.1EUR", "0.05EUR", "0.02EUR", "0.01EUR", "OOD"]
test_folder = r"./data/test/"
valid_extensions = ('.jpg')
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
model = ResNetModel()
model.load_state_dict(torch.load(r"./model/model.pth"))
model.eval()
transform = v2.Compose([
    v2.Resize(224),
    v2.CenterCrop(224),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
file_list = os.listdir(test_folder)
image_files = [f for f in file_list if f.lower().endswith(valid_extensions)]

final_count = []

image_files = sorted(image_files)

for image_file in tqdm(image_files):
    answer = {}
    coins = []
    image_path = os.path.join(test_folder, image_file)
    image = cv2.imread(image_path)
    image_name = image_file.removesuffix(".JPG")
    if image is None:
        print(f"Failed to load {image_file}")
        continue
    
    detected_im = detector(image)
    pathes = []
    coins = []
    image = Image.open(image_path)
    for i, detect_test in enumerate(detected_im):
        img_rgb = image
        d_box = detect_test
        coins.append(image.crop((d_box.left(), d_box.top(), d_box.right(), d_box.bottom())))
    answer["id"] = image_name
    for i in range(16):
        answer[possible_value[i]] = 0

    for c in coins:
        with torch.no_grad():
            coin_input = transform(c).unsqueeze(0).to(device)
            output = model(coin_input)
            res = output.argmax()
            answer[possible_value[res.item()]] += 1
    final_count.append(answer)

print("Creating csv file...")

pandas_df = pd.DataFrame(final_count)
pandas_df.to_csv(r"./data/test-predictions.csv", index=False)



100%|██████████| 162/162 [07:20<00:00,  2.72s/it]

Creating csv file...


In [53]:
# Compare prediction with labels
labels = pd.read_csv(r"./data/train_labels.csv")
labels = labels.sort_values(by="id")
predictions = pd.read_csv(r"./data/train-predictions.csv")
predictions = predictions.sort_values(by="id")
for id in labels["id"]:
    label = labels[labels["id"] == id]
    prediction = predictions[predictions["id"] == id]
    for c in possible_value:
        if label[c].values[0] != prediction[c].values[0]:
            print(f"Error for {id} for {c} label: {label[c].values[0]} prediction: {prediction[c].values[0]}")
            print("")

Error for L1010279 for 2CHF label: 1 prediction: 0

Error for L1010279 for 1CHF label: 0 prediction: 1

Error for L1010287 for 2CHF label: 1 prediction: 0

Error for L1010287 for 0.5CHF label: 0 prediction: 1

Error for L1010294 for 0.2CHF label: 0 prediction: 1

Error for L1010294 for 0.1CHF label: 1 prediction: 0

Error for L1010298 for 2CHF label: 2 prediction: 3

Error for L1010298 for 0.5CHF label: 2 prediction: 1

Error for L1010341 for 1CHF label: 2 prediction: 1

Error for L1010341 for 0.5CHF label: 0 prediction: 1

Error for L1010370 for 0.2CHF label: 1 prediction: 2

Error for L1010370 for 0.1CHF label: 2 prediction: 1

Error for L1010383 for 5CHF label: 0 prediction: 1

Error for L1010383 for 0.1CHF label: 1 prediction: 0

Error for L1010390 for 1CHF label: 0 prediction: 1

Error for L1010390 for 0.5CHF label: 1 prediction: 0

Error for L1010395 for 1CHF label: 0 prediction: 1

Error for L1010395 for 0.5CHF label: 1 prediction: 0

Error for L1010421 for 1CHF label: 1 predict